<h3>Транспортна задача</h3>
<p>Транспортна задача є найбільш популярною задачею лінійного програмування. Найефективнішим методом аналітичного розв’язку є метод північно-західного кута та метод потенціалів.</p>
<p>Автоматизуємо її використовуючи дані методи</p>

In [28]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog

class TransportTask:
    def read_task(self, filename):
        df = pd.read_csv(filename, header=None)

        self.supplies = df.iloc[-1, :-1].to_numpy()
        self.demands = df.iloc[:-1, -1].to_numpy()
        self.costs = df.iloc[:-1, :-1].to_numpy()

    def print_matrix(self):
        print('Supplies: ', self.supplies)
        print('Demands: ', self.demands)
        print('Costs:\n', self.costs)

    def solve(self):
        # Flatten the cost matrix to 1D array
        c = self.costs.flatten()

        # Coefficients for the inequality constraints Ax <= b
        A_eq = np.zeros((len(self.supplies) + len(self.demands), len(c)))
        b_eq = np.zeros(len(self.supplies) + len(self.demands))

        # Constraints for supplies (rows in A_eq)
        for i in range(len(self.supplies)):
            A_eq[i, i * len(self.demands):(i + 1) * len(self.demands)] = 1
            b_eq[i] = self.supplies[i]

        # Constraints for demands (columns in A_eq)
        for j in range(len(self.demands)):
            A_eq[len(self.supplies) + j, j:len(c):len(self.demands)] = 1
            b_eq[len(self.supplies) + j] = self.demands[j]

        # Bounds for decision variables (each variable is non-negative)
        bounds = [(0, None) for _ in range(len(c))]

        # Solve the linear programming problem
        result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

        # Reshape the result to a matrix
        allocation = result.x.reshape(self.costs.shape)

        return allocation

    def north_west(self):
        n_suppliers = len(self.supplies)
        n_customers = len(self.demands)

        allocation = np.zeros((n_suppliers, n_customers))

        i, j = 0, 0

        while i < n_suppliers and j < n_customers:
            quantity = np.min([self.supplies[i], self.demands[j]])
            allocation[i, j] = quantity

            self.supplies[i] -= quantity
            self.demands[j] -= quantity

            if self.supplies[i] == 0:
                i += 1
            if self.demands[j] == 0:
                j += 1

        return allocation


Розв'яжемо задачу використовуючи метод північно-західного кута.

In [29]:
task1 = TransportTask()
task1.read_task('task1.csv')
task1.print_matrix()
result = task1.north_west()
print('Optimal transportation matrix:\n', result)

Supplies:  [140.  90. 160. 110. 150.]
Demands:  [230. 250. 170.]
Costs:
 [[40 19 25 25 35]
 [49 26 27 18 38]
 [46 27 36 40 45]]
Optimal transportation matrix:
 [[140.   0.   0.]
 [ 90.   0.   0.]
 [  0. 160.   0.]
 [  0.  90.  20.]
 [  0.   0. 150.]]


Розв'яжемо задачу використовуючи метод розв'язання задач лінійного програмування бібліотеки scipy

In [30]:
task2 = TransportTask()
task2.read_task('task2.csv')
task2.print_matrix()
result = task2.solve()
print('Optimal transportation matrix:\n', result)

Supplies:  [210. 150. 120. 135. 135.]
Demands:  [300. 250. 200.]
Costs:
 [[ 4  8 13  2  7]
 [ 9  4 11  9 17]
 [ 3 16 10  1  4]]
Optimal transportation matrix:
 [[210.   0.   0.  90.   0.]
 [ 60.   0.   0. 120.   0.]
 [135.   0.   0. 115.  20.]]
